In [ ]:
from google.colab import files
from io import BytesIO
from PIL import Image
uploaded = files.upload()

Saving images.zip to images.zip


In [ ]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
import zipfile
with zipfile.ZipFile("images.zip", 'r') as zip_ref:
    zip_ref.extractall('/images')

User uploaded file "images.zip" with length 40431308 bytes


In [ ]:
import os 
folders=os.listdir("/images/dataset")
print(folders)

['dataset', 'horses', 'cats', 'dogs', 'humans']


In [ ]:
image_data = []
label = []
label_dict ={
    "cats":0,
    "dogs":1,
    "horses":2,
    "humans":3
}

In [ ]:
from keras.preprocessing import image
for ix in folders:
  if ix  != 'dataset':
    path = os.path.join("/images/dataset",ix)
    for im in os.listdir(path):
      img = image.load_img(os.path.join(path,im),target_size=((224,224)))
      img_array = image.img_to_array(img)
      image_data.append(img_array)
      label.append(label_dict[ix])

In [ ]:
print(len(image_data),len(label))

808 808


In [ ]:
import random
combined = list(zip(image_data,label))
random.shuffle(combined)
image_data[:],label[:] = zip(*combined)

In [ ]:
print(label[:5])

[3, 3, 1, 1, 0]


In [ ]:
import numpy as np
X_train = np.array(image_data)
Y_train = np.array(label)
print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808,)


In [ ]:
from keras.utils import np_utils
print(Y_train.shape)
Y_train = np_utils.to_categorical(Y_train)

(808,)


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
import matplotlib.pyplot as plt

In [ ]:
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)

model_new = Model(model.input, outputs=fc2)
model_new.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
adam = Adam(lr=0.00003)
model_new.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f48dc361a20>
1 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f48dc094630>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f48dc0947b8>
3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f48dc094c50>
4 <tensorflow.python.keras.layers.core.Activation object at 0x7f48db7ea4a8>
5 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f48db7ea2b0>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f48d07b35f8>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f48d07bde10>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f48d075cbe0>
9 <tensorflow.python.keras.layers.core.Activation object at 0x7f48d0762b00>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f48d0762978>
11 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization o

In [ ]:
for ix in range(169):
  model_new.layers[ix].trainable = False

hist = model_new.fit(X_train,Y_train,shuffle=True,batch_size=10,epochs=5,validation_split=0.20)

Epoch 1/5
65/65 [==============================] - 9s 135ms/step - loss: 0.4528 - accuracy: 0.8452 - val_loss: 0.1978 - val_accuracy: 0.9444
Epoch 2/5
65/65 [==============================] - 7s 105ms/step - loss: 0.2768 - accuracy: 0.9102 - val_loss: 0.1626 - val_accuracy: 0.9506
Epoch 3/5
65/65 [==============================] - 7s 106ms/step - loss: 0.2199 - accuracy: 0.9427 - val_loss: 0.1460 - val_accuracy: 0.9568
Epoch 4/5
65/65 [==============================] - 7s 106ms/step - loss: 0.1589 - accuracy: 0.9582 - val_loss: 0.1358 - val_accuracy: 0.9568
Epoch 5/5
65/65 [==============================] - 7s 106ms/step - loss: 0.1304 - accuracy: 0.9706 - val_loss: 0.1292 - val_accuracy: 0.9568
